This notebook is intended to demonstrate how select registration, segmentation, and image mathematical methods of ITKTubeTK can be combined to perform multi-channel brain extraction (aka. skull stripping for patient data containing multiple MRI sequences).

There are many other (probably more effective) brain extraction methods available as open-source software such as BET and BET2 in the FSL package (albeit such methods are only for single channel data).   If you need to perform brain extraction for a large collection of scans that do not contain major pathologies, please use one of those packages.   This notebook is meant to show off the capabilities of specific ITKTubeTK methods, not to demonstration how to "solve" brain extraction.

In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [2]:
InputBaseDir = "G:/My Drive/Projects/Proj_UNC_StrokeCollaterals/Experiments/UNC/CTP/CTAT-003"

CTPMaxFilename = InputBaseDir + "-MinMax/max.nrrd"
CTPMinFilename = InputBaseDir + "-MinMax/min.nrrd"
CTPBrainFilename = InputBaseDir + "-MinMax/max-Brain.nrrd"

imMax = itk.imread(CTPMaxFilename, itk.F)
imMin = itk.imread(CTPMinFilename, itk.F)
imBrain = itk.imread(CTPBrainFilename, itk.F)

In [3]:
view(imBrain)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [4]:
ImageType = itk.Image[itk.F, 3]

imMath = ttk.ImageMath.New(Input=imBrain)
imMath.Threshold( 0.00001, 2000, 1, 0)
imMath.Erode(10,1,0)
imBrainMaskErode = imMath.GetOutput()

imMath.SetInput(imMax)
imMath.AddImages(imMin,1,-1)
imDiff = imMath.GetOutput()
imMath.ReplaceValuesOutsideMaskRange(imBrain, 0.0001, 2000, 0)
imDiffBrain = imMath.GetOutput()
imMath.ReplaceValuesOutsideMaskRange(imBrainMaskErode, 0.5, 1.5, 0)
imDiffBrainErode = imMath.GetOutput()


In [5]:
tmpA = itk.GetArrayViewFromImage(imDiffBrain)
tmpAE = itk.GetArrayViewFromImage(imDiffBrainErode)
zMax = tmpA.shape[0]
clip = 0
while((np.amax(tmpA[clip:clip+1,:,:])>1000) | (np.amax(tmpA[clip:clip+1,:,:])==0)):
    clip += 1
if(clip>0):
    tmpA[0:clip,:,:]=0
    tmpAE[0:clip,:,:]=0
clip = 1
while((np.amax(tmpA[zMax-clip:zMax-clip+1,:,:])>1000) | (np.amax(tmpA[zMax-clip:zMax-clip+1,:,:])==0)):
    clip += 1
print(clip, np.amax(tmpA[zMax-clip:zMax-clip+1,:,:]))
clip = clip - 1
if(clip>0):
    tmpA[zMax-clip:zMax,:,:]=0  #Happens to imDiffBrain since this array is a view of an itk image
    tmpAE[zMax-clip:zMax,:,:]=0  #Happens to imDiffBrain since this array is a view of an itk image

1 588.5109


In [6]:
view(imDiffBrain)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [7]:
imMath = ttk.ImageMath[ImageType,ImageType].New()
imMath.SetInput(imDiffBrainErode)
imMath.Blur(1.5)
imBlur = imMath.GetOutput()
imBlurArray = itk.GetArrayViewFromImage(imBlur)

numSeeds = 15
seedCoverage = 20
seedCoord = np.zeros([numSeeds,3])
for i in range(numSeeds):
    seedCoord[i] = np.unravel_index(np.argmax(imBlurArray, axis=None), imBlurArray.shape)
    indx = [int(seedCoord[i][0]),int(seedCoord[i][1]),int(seedCoord[i][2])]
    minX = max(indx[0]-seedCoverage,0)
    maxX = max(indx[0]+seedCoverage,imBlurArray.shape[0])
    minY = max(indx[1]-seedCoverage,0)
    maxY = max(indx[1]+seedCoverage,imBlurArray.shape[1])
    minZ = max(indx[2]-seedCoverage,0)
    maxZ = max(indx[2]+seedCoverage,imBlurArray.shape[2])
    imBlurArray[minX:maxX,minY:maxY,minZ:maxZ]=0
    indx.reverse()
    seedCoord[:][i] = imDiffBrain.TransformIndexToPhysicalPoint(indx)
print(seedCoord)

[[  -7.00160217 -163.7144165   304.47746277]
 [  -4.23602295 -145.46159363  297.28695679]
 [ -18.61703491 -205.7512207   301.71188354]
 [ -19.7232666  -186.94528198  292.30891418]
 [  -5.34225464 -138.82420349  285.67152405]
 [ -40.18855286 -212.94172668  296.73384094]
 [ -51.8039856  -207.96368408  297.28695679]
 [ -25.80754089 -144.90847778  278.48101807]
 [ -48.48529053 -193.58267212  305.03057861]
 [  12.35745239 -176.98919678  266.86558533]
 [ -44.61347961 -181.96723938  309.45550537]
 [ -37.42297363 -164.26753235  296.1807251 ]
 [ -56.22891235 -181.96723938  309.45550537]
 [  -2.57667542 -142.6960144   266.86558533]
 [ -58.99449158 -155.97079468  292.30891418]]


In [8]:
# Manually extract a few vessels to form an image-specific training set
vSeg = ttk.SegmentTubes.New(Input=imDiffBrain)
vSeg.SetVerbose(True)
vSeg.SetMinRoundness(0.4)
vSeg.SetMinCurvature(0.002)
vSeg.SetRadiusInObjectSpace( 1 )
for i in range(numSeeds):
    print("**** Processing seed " + str(i) + " : " + str(seedCoord[i]))
    vSeg.ExtractTubeInObjectSpace( seedCoord[i], i )
    
tubeMaskImage = vSeg.GetTubeMaskImage()

**** Processing seed 0 : [  -7.00160217 -163.7144165   304.47746277]
**** Processing seed 1 : [  -4.23602295 -145.46159363  297.28695679]
**** Processing seed 2 : [ -18.61703491 -205.7512207   301.71188354]
**** Processing seed 3 : [ -19.7232666  -186.94528198  292.30891418]
**** Processing seed 4 : [  -5.34225464 -138.82420349  285.67152405]
**** Processing seed 5 : [ -40.18855286 -212.94172668  296.73384094]
**** Processing seed 6 : [ -51.8039856  -207.96368408  297.28695679]
**** Processing seed 7 : [ -25.80754089 -144.90847778  278.48101807]
**** Processing seed 8 : [ -48.48529053 -193.58267212  305.03057861]
**** Processing seed 9 : [  12.35745239 -176.98919678  266.86558533]
**** Processing seed 10 : [ -44.61347961 -181.96723938  309.45550537]
**** Processing seed 11 : [ -37.42297363 -164.26753235  296.1807251 ]
**** Processing seed 12 : [ -56.22891235 -181.96723938  309.45550537]
**** Processing seed 13 : [  -2.57667542 -142.6960144   266.86558533]
**** Processing seed 14 : [ -5

In [9]:
imMath.SetInput(tubeMaskImage)
imMath.AddImages(imDiffBrain, 200, 1)
blendIm = imMath.GetOutput()
view(blendIm)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [10]:
LabelMapType = itk.Image[itk.UC,3]

trMask = ttk.ComputeTrainingMask[ImageType,LabelMapType].New()
trMask.SetInput( tubeMaskImage )
trMask.SetGap( 4 )
trMask.SetObjectWidth( 1 )
trMask.SetNotObjectWidth( 1 )
trMask.Update()
fgMask = trMask.GetOutput()

In [11]:
view(fgMask)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [12]:
enhancer = ttk.EnhanceTubesUsingDiscriminantAnalysis[ImageType,LabelMapType].New()
enhancer.AddInput( imDiff )
enhancer.SetLabelMap( fgMask )
enhancer.SetRidgeId( 255 )
enhancer.SetBackgroundId( 128 )
enhancer.SetUnknownId( 0 )
enhancer.SetTrainClassifier(True)
enhancer.SetUseIntensityOnly(True)
enhancer.SetScales([0.43,1.29,3.01])
enhancer.Update()
enhancer.ClassifyImages()

In [13]:
im1vess = itk.SubtractImageFilter( Input1=enhancer.GetClassProbabilityImage(0), Input2=enhancer.GetClassProbabilityImage(1))

imMath.SetInput(imDiffBrain)
imMath.Threshold(0.0001,2000,1,0)
imMath.Erode(2,1,0)
imBrainE = imMath.GetOutput()

imMath.SetInput(im1vess)
imMath.ReplaceValuesOutsideMaskRange(imBrainE, 1, 1, -0.001)
im1vessBrain = imMath.GetOutput()
#view(enhancer.GetClassProbabilityImage(0))
view(im1vessBrain)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [14]:
itk.imwrite( im1vess, InputBaseDir + "-MinMax/diff-VesselEnhanced.nrrd", compression=True)

itk.imwrite( im1vessBrain, InputBaseDir + "-MinMax/Brain-VesselEnhanced.nrrd", compression=True)